In [1]:
import geopandas as gpd

lidar2 = gpd.read_file('../descarga_lidar/FINALES/NAVARRA_LIDA3_25830/NAVARRA_LIDA3_25830.shp')
zonas = gpd.read_file('shp/zonas_estudio.shp')

In [2]:
zonas.total_bounds

array([ 646661.24463813, 4721164.2202975 ,  648280.34732985,
       4722060.77378944])

In [3]:
from shapely import box

b = box(*list(zonas.total_bounds))

In [4]:
files = lidar2[lidar2.intersects(b)]

In [8]:
import requests
import os

url = 'https://centrodedescargas.cnig.es/CentroDescargas/descargaDir'

for i, row in files.iterrows():
    if row['nombre'] in os.listdir('data/lida3/'):
        pass
    else:
        with requests.post( url, data={'secuencialDescDir': row['num']}, stream=True ) as r:
          with open(f'data/lida3/{row['nombre']}', 'wb') as f:
              f.write(r.content)

In [2]:
import pdal

def get_pipeline(src, filename, table_name):
    return f'''{{
      "pipeline":[
        {{
          "type":"readers.las",
          "filename":"data/{src}/{filename}"
        }},
        {{
          "type":"filters.splitter",
          "length":"25"
        }},
        {{
          "type":"writers.pgpointcloud",
          "connection":"host='172.17.0.2' dbname='pointclouds' user='postgres' password='ventanuco' port='5432'",
          "table":"{table_name}",
          "compression":"dimensional",
          "srid":"25830"
        }}
      ]
    }}'''

In [5]:
import pdal

pipeline = pdal.Pipeline(get_pipeline('CLASIFICADA', 'YS-20251017_Vuelo_2_02_zona_03.laz', 'vuelo2_zona2_clas'))
pipeline.execute()

76026640

In [13]:
import os

for row in os.listdir('data/lida3'):
    print(row)
    pipeline = pdal.Pipeline(get_pipeline('lida3', row))
    pipeline.execute()

PNOA-2024-PNR-648-4722-NPC01.laz
PNOA-2024-PNR-647-4723-NPC01.laz
PNOA-2024-PNR-648-4723-NPC01.laz
PNOA-2024-PNR-646-4723-NPC01.laz
PNOA-2024-PNR-646-4722-NPC01.laz
PNOA-2024-PNR-647-4722-NPC01.laz


In [15]:
import psycopg2

conn = psycopg2.connect(database="pointclouds",
                        host="172.17.0.2",
                        user="postgres",
                        password="ventanuco",
                        port="5432")

In [18]:
def get_sql_text(src, zona):
    return f'''
    with patches as (
    select id, pa from {src}
    where pc_intersects(pa, (select geom from zonas_estudio where id = {zona}))
    )
    select id, pc_intersection(pa, (select geom from zonas_estudio where id = {zona}))::pcpatch(3) as pa
    into {src}_zona{zona} from patches'''

In [20]:
for i in [1]:
    # try:
        cursor = conn.cursor()
        cursor.execute(get_sql_text('lida3', i))
        conn.commit()
    # except:
    #     cursor.execute("ROLLBACK")
    #     conn.commit()
    #     print('Ha habido un error')